# COVID-19 cases

Data comes from the ECDC.

Links 

* https://www.ecdc.europa.eu/en/publications-data/download-todays-data-geographic-distribution-covid-19-cases-worldwide
* https://www.kaggle.com/pavlofesenko/interactive-titanic-dashboard-using-bokeh
* https://stackoverflow.com/questions/41382310/adding-a-second-y-axis-in-bokehjs

In [1]:
import pandas as pd
import numpy as np
import matplotlib as mpl
import pylab as plt
pd.set_option('display.width',140)
from bokeh.io import output_file, show, output_notebook, export_png
from bokeh.models import ColumnDataSource, ColorBar, HoverTool, Legend, DataTable, TableColumn
from bokeh.tile_providers import get_provider, Vendors
from bokeh.plotting import figure
from bokeh.palettes import brewer
from bokeh.layouts import row, column, gridplot
from bokeh.models import CustomJS, Slider, Select, Plot, Button, LinearAxis, Range1d, DatetimeTickFormatter
from bokeh.models.glyphs import Line, MultiLine
from bokeh.palettes import Category10
output_notebook()
#output_file('test.html')

Loading BokehJS ...

In [16]:
#df = pd.read_csv('ecdc_data.csv')
df = pd.read_excel('https://www.ecdc.europa.eu/sites/default/files/documents/COVID-19-geographic-disbtribution-worldwide.xlsx')
df['dateRep'] = pd.to_datetime(df.dateRep, infer_datetime_format=True)
df = df.sort_values(['countriesAndTerritories','dateRep'])
df['cumcases'] = df.groupby(['countriesAndTerritories'])['cases'].apply(lambda x: x.cumsum())
data = pd.pivot_table(df,index='dateRep',columns='countriesAndTerritories',values='cumcases').reset_index()
#data = data.fillna(0)
summary = df.groupby('countriesAndTerritories').agg({'deaths':np.sum,'cases':np.sum}).reset_index()

In [17]:
summary

,countriesAndTerritories,deaths,cases
0,Afghanistan,3,106
1,Albania,10,197
2,Algeria,26,409
3,Andorra,4,308
4,Angola,0,4
...,...,...,...
191,Uzbekistan,1,133
192,Venezuela,1,119
193,Vietnam,0,223
194,Zambia,0,28


In [14]:
df[:5]

,dateRep,day,month,year,cases,deaths,countriesAndTerritories,geoId,countryterritoryCode,popData2018,cumcases
78,2019-12-31,31,12,2019,0,0,Afghanistan,AF,AFG,37172386.0,0
77,2020-01-01,1,1,2020,0,0,Afghanistan,AF,AFG,37172386.0,0
76,2020-01-02,2,1,2020,0,0,Afghanistan,AF,AFG,37172386.0,0
75,2020-01-03,3,1,2020,0,0,Afghanistan,AF,AFG,37172386.0,0
74,2020-01-04,4,1,2020,0,0,Afghanistan,AF,AFG,37172386.0,0


In [4]:
def bokeh_plot_cases():
    """Plot cases per country"""
    
    countries = ['Ireland','Sweden']
    p = figure(plot_width=500,plot_height=300,x_axis_type='datetime',tools="hover,xwheel_zoom,xpan")        
    source = ColumnDataSource(data)
    colors = Category10[10] + Category10[10]
    i=0
    items=[]
    for c in countries:
        line = Line(x='dateRep',y=c, line_color=colors[i],line_width=3,name=c)
        glyph = p.add_glyph(source, line)
        i+=1
        items.append((c,[glyph]))
    p.add_layout(Legend(
            location="top_left",
            items=items))        
    p.grid.visible = False
    return p

names = list(df.countriesAndTerritories.unique() )
select = Select(title="Option:", value=names[0], options=names)
#select.on_change('value', callback)
plot = bokeh_plot_cases() 
g = gridplot([[select,plot]], toolbar_location='below')
show(g)


In [18]:
source = ColumnDataSource(data)

#print (filt_data)
# create CDS for filtered sources
filt_data = data[['dateRep','China']].rename(columns={'China':'cases'})
src2 = ColumnDataSource(filt_data)
filt_data = data[['dateRep','Ireland']].rename(columns={'Ireland':'cases'})
src3 = ColumnDataSource(filt_data)

hover_tool = HoverTool(tooltips=[
            ('Cases', '@cases'),
            ('Date', '@dateRep{%F}')],
            formatters={'dateRep': 'datetime'}
        )

p1 = figure(plot_width=600,plot_height=400,x_axis_type='datetime',
           tools=[hover_tool],title='country 1')
#p1 = Plot(title = None, plot_width = 400, plot_height = 400, x_axis_type='datetime',)
p1.line(x='dateRep',y='cases', source=src2, legend_label="country 1", line_color='blue',
        line_width=3,line_alpha=.8)
p1.extra_y_ranges = {"y2": Range1d(start=-50, end=filt_data.cases.max()+50)}
p1.add_layout(LinearAxis(y_range_name="y2", axis_label='right'), 'right')
p1.line(x='dateRep',y='cases', source=src3, legend_label="country 2", line_color='orange',
        line_width=3,line_alpha=.8,y_range_name="y2")
p1.legend.location = "top_left"
p1.xaxis.axis_label = 'Date'
p1.xaxis.formatter=DatetimeTickFormatter(days="%d/%m",
months="%m/%d %H:%M",
)

#p2 = figure(plot_width=300,plot_height=400,
#           tools="hover,xwheel_zoom,xpan",title='pie')
#p2.wedge(x=0, y=0, radius=0.8, source=src2,
#         start_angle=cumsum('angle', include_zero=True), end_angle=cumsum('angle'))

print (p1.extra_y_ranges['y2'])
code="""
c = cb_obj.value;
console.log(c);
var y = s1.data[c];
console.log(y);
s2.data['cases'] = y;
y_range.start = 0;
y_range.end = Math.max(y);
s2.change.emit();
"""
callback1 = CustomJS(args=dict(s1=source,s2=src2,y_range=p1.y_range), code=code)
callback2 = CustomJS(args=dict(s1=source,s2=src3,y_range=p1.extra_y_ranges['y2']), code=code)
names = list(df.countriesAndTerritories.unique() )
names_sub=['China','United_Kingdom','United_States_of_America','Spain','Italy','France','Australia','Ireland','Sweden']
select1 = Select(title="Country 1:", value='China', options=names_sub)
select1.js_on_change('value', callback1)
select2 = Select(title="Country 2:", value='Ireland', options=names)
select2.js_on_change('value', callback2)
btn = Button(label='Update')

layout = column(row(select1,select2), row(p1))
show(layout)


Range1d(id='5071', ...)


In [ ]:
#output_file('test.html')
#show(layout)